## Importações necessárias

In [ ]:
import pandas as pd
import os
from geobr import read_municipality
import string
import re
import unidecode
import numpy as np
from IPython.display import clear_output 
from os import system, name 
from time import sleep 

###Função que busca uma determinada palavra em uma string (se não retorna "None" é porque a palavra existe) 
def achar_palavra(palavra):
    return re.compile(r'\b({0})\b'.format(palavra), flags=re.IGNORECASE).search


## Atenção
    Defina na célula a seguir em quais semanas será realizada a correção das localizações

In [ ]:
print("Iniciando correção...")

### Recuperando informações oficiais sobre as cidades do Brasil

In [ ]:
#Dataframe com os municípios oficiais do Brasil e respectivos estados
geobr_mun = read_municipality(code_muni="all", year=2019)[["code_muni","name_muni","abbrev_state"]]

#corrige o formato da coluna code_muni, para que os códigos dos municípios sejam inteiros
geobr_mun = geobr_mun.astype({'code_muni':int})

#Transforma o cód do município no index do dataframe
geobr_mun.set_index('code_muni', inplace=True)

#geobr_mun.tail(3)

In [ ]:
#função util adiante para tratar abreviações em nomes de cidades. Ex: "Machadinho D'oeste" -> "Machadinho do Oeste"
def desabrevia_dos(texto):
    for abrev in ["D'", "d'"]:
        if abrev in texto:
            texto = texto.replace(abrev,"do ")
    return texto


def adequa_base(cidade):
    nome_desabreviado = desabrevia_dos(cidade).lower() #desabrevia e deixa em minúsculo
    nome_sem_acentos = unidecode.unidecode(nome_desabreviado) #retirar acentos
    return nome_sem_acentos


serie_cidades_brasil = geobr_mun['name_muni']
for i, item in serie_cidades_brasil.iteritems():
    serie_cidades_brasil.loc[i] = adequa_base(item)

    

### Recuperando as localidades coletadas do Twitter

In [ ]:
#Dataframe com os municípios coletados do twitter em cada semana
#path = os.getcwd() + '\\results\\week_'
#filename = 'tweets_locations_' 
path = os.getcwd() + '/results/week_'
filename = 'tweets_locations_'

df_locations = pd.DataFrame(None)

#lê os arquivos das semanas definidas no início
for i in range(1,38):#(1,38)
    clear_output()
    print(f'Acessando semana {i}')
    tempDF = pd.read_csv(path + str(i) + '\\' + filename + str(i) + '.csv', sep=';') #<-no linux, comente essa linha e descomente a de baixo
    #tempDF = pd.read_csv(path + str(i) + '/' + filename + str(i) + '.csv', sep=';') 
    tempDF = tempDF.assign(week=i)
    df_locations = pd.concat([df_locations, tempDF])


#Armaneza em outro df e exclui do original linhas com entradas nulas em alguma das colunas ('location' ou 'count')    
df_entradas_nulas = df_locations[df_locations.isnull().any(axis=1)]
df_entradas_nulas.reset_index(drop=True,inplace=True)
df_locations.dropna(inplace=True)  


#Armaneza em outro df e exclui do original linhas com entradas que contém apenas espaços em branco na coluna 'location'
df_entradas_em_branco = df_locations[df_locations['location'].apply(lambda x: True if str.isspace(x) else False)]
df_entradas_em_branco.reset_index(drop=True,inplace=True)
df_locations = df_locations[df_locations['location'].apply(lambda x: False if str.isspace(x) else True)]


#conserta os índices
df_locations.reset_index(drop=True,inplace=True)

#DESCRIÇÃO: Contagem de quantos tweets com determinada localização apareceram em determinada semana
#Obs: se o mesmo usuário fez mais de um tweets, a localização dele aparecerá mais de uma vez
#df_locations.head(5)

### Corrigindo os textos dos tweets coletados 

In [ ]:
#função para tratamento do texto contido no campo "location" nos tweets coletados
#Fonte: https://www.analyticsvidhya.com/blog/2020/11/text-cleaning-nltk-library/
#fonte: https://www.geeksforgeeks.org/python-efficient-text-data-cleaning/

def trata_texto(texto_entrada):
    
    #Varias localidades vem no formato "cidade, estado" e estamos interessados só na cidade
    texto_entrada = texto_entrada.split(",")[0]
        
    #
    texto_sem_hiperlinks = re.sub(r'https?:\/\/.\S+', "", texto_entrada) #pode deixar espaços extra
    
    #
    texto_sem_mais_e_hifen = re.sub("\+"," ",re.sub("\-"," ", texto_sem_hiperlinks)) #prejudica hiperlinks
    
    #
    texto_sem_hash = re.sub(r'#', '', texto_sem_mais_e_hifen)
    
    #verifica se "do" está abreviado no texto e corrige .Ex: "Machadinho D'oeste" -> "Machadinho do oeste"
    texto_desabreviado = desabrevia_dos(texto_sem_hash)
    
    #    
    texto_sem_outras_pontuacoes = "".join([i for i in texto_desabreviado if i not in string.punctuation])
    
    #Atenção -> só consegue separar palavras onde as duas comecem com maiúscula, como em SãoPaulo, mas não em Sãopaulo
    texto_sem_palavras_juntas = " ".join([s for s in re.split("([A-Z][a-z]+[^A-Z]*)",texto_sem_outras_pontuacoes) if s])  
    
    #
    texto_sem_espaços_extra = re.sub("\s+"," ", texto_sem_palavras_juntas)
    
    #
    texto_em_minusculo = texto_sem_espaços_extra.lower()
    
    #
    texto_sem_acentos = unidecode.unidecode(texto_em_minusculo)
    
    
    return texto_sem_acentos

#texto de teste
#trata_texto ("MaisNotícias D'oeste: você, pode ---- encontrar,, mais #informações em https://www.analyticsvidhya.com/blog/2020/11/text-cleaning-nltk-library/ ou em outros lugares como em SãoPaulo+SP. 🍀🌻")    

### Corrigindo a localizações dos tweets
    O processo será realizado a partir dos seguintes dados:
    - geobr_mun ---> dataframe que contém os dados oficiais dos municípios do Brasil
    - dict_cidades ----> dicionário com as mesmas cidades de geobr_mun, mas em um formato facilmente identificável
    - df_locations_tratado ----> dataframe com as localizações dos tweets já limpo

In [ ]:
#criar função que recebe o valor de uma celula da coluna location e tenta localizar ocorrências

#Essa função é demorada
def corrigir_cidades(localizacao_do_tweet):
    #----------------------Identifica possíveis locais e guarda em um dicionário
    locais_identificados = {}
    for key, item in serie_cidades_brasil.items(): #<--------------gargálo aqui!
        if achar_palavra(item)(localizacao_do_tweet) != None:
            locais_identificados[item] = key #locais_identificados = {cidade:cód}   
    
    #----------------------Seleciona o local com nome maior 
    if locais_identificados != {}:
        maior_nome_cidade = sorted(list(locais_identificados.keys()), key=len)[-1]
        index_do_maior = locais_identificados[maior_nome_cidade]
        
        dicio = geobr_mun.loc[[index_do_maior]].to_dict('records')[0]
        dicio['code_muni'] = index_do_maior         
        return dicio
    else:
        return ('')

In [ ]:
#função para limpar o output (não afeta o resultado)
# define our clear function 
def clear(): 
  
    # for windows 
    if name == 'nt': 
        _ = system('cls') 
        
    # for mac and linux(here, os.name is 'posix') 
    else: 
        _ = system('clear') 

### Atenção!
    A célula abaixo realiza a identificação das cidades, armazenando o resultado no dataframe df_locations_tratado
    O PROCESSO PODE LEVAR ALGUMAS (OU MUITAS) HORAS!!!

In [ ]:
#df_locations = df_locations.sample(10) sample para teste

tam_dataframe = len(df_locations)
contador = 1

for i, row in df_locations.iterrows():
    clear()
    clear_output()
    print(f'{contador} de {tam_dataframe} linhas analisadas ({(contador/tam_dataframe)*100}%)')
    
    dict_cidade_corrigida = corrigir_cidades(trata_texto(row['location'])) #<<<---gargálo na função corrigir_cidades
                                                          
    if (dict_cidade_corrigida!=''):
        
        print(dict_cidade_corrigida['name_muni'])
        
        df_locations.loc[[i], ['name_muni']] = dict_cidade_corrigida['name_muni']
        df_locations.loc[[i], ['abbrev_state']] = dict_cidade_corrigida['abbrev_state']
        df_locations.loc[[i], ['code_muni']] = dict_cidade_corrigida['code_muni']

    #if (contador==15):
    #    break;
    
    contador+=1

df_locations.reset_index(inplace=True, drop=True)
df_locations.to_csv(f'tweets_location_corrigido.csv', index=False)